In [ ]:
# Importeer benodigde modules
from datetime import datetime
from pathlib import Path
import shutil
import time
import os

# Bestanden Hernoemen
Dit script vereenvoudigt het hernoemen van bestanden op basis van projectvariabelen. Het script gebruikt de mapstructuur en gebruikersinput om alle bestanden in een map te hernoemen. Dit zorgt voor duidelijkheid tijdens het annoteren en beoordelen van resultaten, en is noodzakelijk voor het gebruik van meerdere scripts binnen deze repository. Het is het gemakkelijkst om je bestanden te hernoemen voordat je foto's segmenteert of annoteert.

Het script werkt op basis van het opgegeven pad. Wanneer de inputlocatie "ProjectNaam" is, worden alle bestanden in "ProjectNaam" en de onderliggende mappen meegenomen. Dit betekent dat met MapSelectie = [0, -2] een bestand gevonden op locatie:
"c:\gebruiker\python\script\\<b>ProjectNaam\variabel1\variabel2\variabel3\Bestand1.png</b>" 

hernoemd zal worden als "Bestand1-variabel1-variabel2.png" terwijl een ander bestand gevonden op: "c:\gebruiker\python\script\\<b>ProjectNaam\variabel1\variabel2\variabel3\variabel4\Bestand2.png</b>" 

hernoemd zal worden als "Bestand2-variabel1-variabel3.png"

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input mappen met de foto's die hernoemd gaan worden.
InputLocaties : list[Path] = [Path(r"pad/naar/jouw/folder"), Path(r"pad/naar/een/andere/folder")]
# Bijvoorbeeld: [Path(r"pad/naar/jouw/folder"), Path(r"pad/naar/een/andere/folder")]

# Bestandstypes van de doelbestanden. Bestanden met andere types worden genegeerd door het script.
InputBestandsTypes : list[str] = [".jpg", ".png", "jpeg"] 
# Bijvoorbeeld: [".jpg", ".txt", ".docx"]

# Output map waar de hernoemde bestanden worden opgeslagen. Als de map nog niet bestaat, wordt deze aangemaakt.
OutputFolder: Path = Path(r"FolderNaam")
# Bijvoorbeeld: Path(r"pad/naar/jouw/folder") OF Path(r"FolderNaam")

# Het scheidingsteken dat wordt gebruikt in de naam tussen de variabelen. 
# ScheidingsTeken = "-" zou er dus zo uit kunnen zien "0002-A52-flits-1_0_0"
ScheidingsTeken: str = "-" 
# Bijvoorbeeld: "-" of "_" of " " etc.

# Welke mappen worden meegenomen in de nieuwe naam. 
# Positieve waardes (beginnend bij 0) worden gerekend vanaf InputLocaties, negatieve (beginnend bij -1) vanaf de bestandslocatie.
# De volgorde van MapSelectie wordt aangehouden.
MapSelectie: list[int] = [0, -1, 2]
# Bijvoorbeeld: [0, -1, 2]

# Bepaal of je de originele bestandsnaam in de nieuwe naam wilt behouden.
# Als BehoudBestandNaam False is, wordt een "ID000000000000" gebruikt in plaats van de originele naam.
BehoudBestandNaam: bool = True 
# Bijvoorbeeld: True OF False

# Bepaal of je de mapstructuur op de outputlocatie wilt behouden.
BehoudMapStructuur: bool = False 
# Bijvoorbeeld: True OF False

---
## **Code**

In [ ]:
# Verwijder de output map als deze al bestaat
if os.path.exists(OutputFolder):
    shutil.rmtree(OutputFolder)

# Maakt een variabele voor de huidige tijd om te gebruiken als ID. Word ook gebruikt om te checken of de tijden daarmee het ID uniek is.
CurrentTime = datetime.now()

# Zet de bestandstypes in lowercase om hoofdlettergevoeligheid te voorkomen.
InputBestandsTypesLowercase = tuple(file_type.lower() for file_type in InputBestandsTypes)

# loopt door alle bestanden in de inputlocatie(s).
for BaseDir in InputLocaties:
    for root, dirs, files in os.walk(BaseDir):
        for PotentialFiles in files:
            # Check of het bestandstype overeenkomt met de bestandstypes die we willen hernoemen.
            if PotentialFiles.lower().endswith(InputBestandsTypesLowercase):
                # Bepaal mappen van het bestand tot inputlocatie zodat alleen deze mappen worden gebruiken in de nieuwe naam.
                RelativePath = os.path.relpath(root, BaseDir)
                RelativePathParts = RelativePath.split(sep=os.path.sep)

                NieuweBestandsnaam = []
                # Selecteerd de mappen op volgorden zoals die gebruikt gaan worden in de nieuwe naam.
                for Mappen in MapSelectie:
                    NieuweBestandsnaam.append(RelativePathParts[Mappen])
                
                # voegt of de originele bestandsnaam of een ID (op basis van de tijd) toe aan de nieuwe bestandsnaam.
                if BehoudBestandNaam:
                    NieuweBestandsnaam.insert(0, os.path.splitext(PotentialFiles)[0])
                else:
                    # Controleer of de Tijd/ID uniek is anders wacht tot dit wel zo is. 
                    if datetime.now() == CurrentTime:
                        time.sleep(0.000001)
                    
                    # update CurrentTime voor de volgende iteratie.
                    CurrentTime = datetime.now()

                    # Formateer de tijd naar een string en voeg toe aan bestandsnaam.
                    FormattedTime = CurrentTime.strftime("ID%H%M%S%f")
                    NieuweBestandsnaam.insert(0, FormattedTime)
                
                # Bepaal de locatie waar het bestand wordt opgeslagen en zorg dat deze bestaat.
                if BehoudMapStructuur:
                    OpslagLocatie = os.path.join(OutputFolder, RelativePath)
                else:
                    OpslagLocatie = OutputFolder
                
                if not os.path.exists(OpslagLocatie):
                    os.makedirs(OpslagLocatie)

                #Sla bestand op onder nieuwe naam
                OrigineleLocatie = os.path.join(root, PotentialFiles)
                NieuweLocatie = os.path.join(OpslagLocatie, f"{ScheidingsTeken.join(NieuweBestandsnaam)}{os.path.splitext(PotentialFiles)[1]}")
                
                shutil.copyfile(OrigineleLocatie, NieuweLocatie)